In [2]:
import pandas as pd

url = 'https://dataverse.nl/api/access/datafile/354098'
pwt1001 = pd.read_stata(url)

countries = ['Australia', 'Austria',  'Belgium', 'Canada', 'Denmark', 'Finland', 'France', 'Germany', 'Greece', 'Iceland', 'Ireland', 'Italy', 'Japan', 'Netherlands', 'New Zealand', 'Norway', 'Portugal', 'Spain', 'Sweden', 'Switzerland', 'United Kingdom', 'United States' ]
data = pwt1001.loc[pwt1001['country'].isin(countries)][['year', 'countrycode', 'country', 'rgdpna', 'rnna', 'emp', 'avh', 'labsh', 'hc', 'rtfpna']]
data = data.loc[(data['year'] >= 1990) & (data['year'] <= 2019)].dropna() 

data['alpha'] = 1 - data['labsh']
data['gdp_per_worker'] = data['rgdpna'] / data['emp']
data['capital_per_worker'] = data['rnna'] / data['emp']
data['human_capital_per_worker'] = data['hc']
data['hours_per_worker'] = data['avh'] 

def calculate_average_growth_rate(data, start_year, end_year, column):
    start_value_data = data.loc[data['year'] == start_year, column]
    end_value_data = data.loc[data['year'] == end_year, column]
    
    if start_value_data.empty or end_value_data.empty:
        return float('nan')  # データが存在しない場合、NaNを返す
    
    start_value = start_value_data.iloc[0]
    end_value = end_value_data.iloc[0]
    
    years = end_year - start_year
    average_growth_rate = (end_value / start_value) ** (1 / years) - 1
    
    return average_growth_rate

results = []
for country in countries:
    country_data = data[data['country'] == country]
    alpha = country_data['alpha'].mean()
    
    gdp_growth = calculate_average_growth_rate(country_data, 1990, 2019, 'gdp_per_worker')
    capital_deepning = calculate_average_growth_rate(country_data, 1990, 2019, 'capital_per_worker') * alpha
    hc_growth = calculate_average_growth_rate(country_data, 1990, 2019, 'human_capital_per_worker')
    hours_growth = calculate_average_growth_rate(country_data, 1990, 2019, 'hours_per_worker')
    
    labor_growth = (1 - alpha) * (hc_growth + hours_growth)
    tfp_growth = gdp_growth - capital_deepning - labor_growth
    
    tfp_share = tfp_growth / gdp_growth if gdp_growth != 0 else float('nan')
    capital_share = capital_deepning / gdp_growth if gdp_growth != 0 else float('nan')
    labor_share = labor_growth / gdp_growth if gdp_growth != 0 else float('nan')
    
    results.append({
        'Country': country,
        'Growth Rate': gdp_growth,
        'TFP Growth': tfp_growth,
        'Capital Deepning': capital_deepning,
        'Labor Growth': labor_growth,
        'TFP Share': tfp_share,
        'Capital Share': capital_share,
        'Labor Share': labor_share,
    })


growth_accounting_table = pd.DataFrame(results)

average_values = {
    'Country': 'Average',
    'Growth Rate': growth_accounting_table['Growth Rate'].mean(),
    'TFP Growth': growth_accounting_table['TFP Growth'].mean(),
    'Capital Deepning': growth_accounting_table['Capital Deepning'].mean(),
    'Labor Growth': growth_accounting_table['Labor Growth'].mean(),
    'TFP Share': growth_accounting_table['TFP Share'].mean(),
    'Capital Share': growth_accounting_table['Capital Share'].mean(),  # 修正点
    'Labor Share': growth_accounting_table['Labor Share'].mean(),
}

average_values_df = pd.DataFrame([average_values])

growth_accounting_table = pd.concat([growth_accounting_table, average_values_df], ignore_index=True)

print(growth_accounting_table.to_string(index=False, float_format='{:.2%}'.format))





       Country  Growth Rate  TFP Growth  Capital Deepning  Labor Growth  TFP Share  Capital Share  Labor Share
     Australia        1.28%       0.95%             0.41%        -0.07%     73.96%         31.91%       -5.86%
       Austria        1.04%       0.46%             0.56%         0.01%     44.44%         54.23%        1.33%
       Belgium        0.95%       0.15%             0.61%         0.19%     16.07%         63.90%       20.03%
        Canada        0.98%       0.31%             0.52%         0.14%     31.69%         53.64%       14.67%
       Denmark        1.36%       0.63%             0.50%         0.23%     46.25%         36.71%       17.04%
       Finland        1.45%       0.61%             0.65%         0.18%     42.40%         45.19%       12.42%
        France        0.91%       0.35%             0.45%         0.12%     38.16%         49.18%       12.65%
       Germany        1.04%       0.72%             0.47%        -0.14%     68.78%         44.56%      -13.34%
 

#考察・感想・メモ 
日本のTFP Growthは海外と比べると低い水準ではあるが、圧倒的に劣っているかというとそういうわけでもない。一人当たりGDPで日本よりも上位に位置するドイツやアメリカと大きく差があるのは、TFP Shareなのではないかと思う。日本はLabor Shareが減っていてかつCapital Shareが他国より高い一方、TFP Shareは低い水準にある。つまり、資本への代替は進んでいるし、機械等を使って効率を高めることはできているのだろうが、TFPの生産への寄与が少ない、つまり技術革新や効率改善が経済成長に寄与できていないと考えられる。資本や資本への投資はそれなりにできているのに、技術革新がそれについてこないというのは、要するに資本の使い方が下手で、十分に使いこなせていないということになるのだろう。ただ、設定している関数を見ればわかるが、これらはあくまでGDP（Y）の変動を資本、労働、その他（TFP）の三つで分解しているものでしかないので、TFP＝技術革新と考えるのは早計であるともいえる。（TFPはあくまで資本・労働以外の、Yの変化の要因をごちゃまぜにしたものでしかない）
